In [119]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math

#Kerry added these for her own benefit
import matplotlib.pyplot as plt
from treelib import Tree, Node
from collections import Counter

### Create Decision Tree
1. Find the best reatures and threshold 
2. Find the dominant class label in the node 
3. Calculating entropy 
4. Predict function 
5. Display the nodes and trees 

In [130]:
def calcEntropy(lst, values): #lst has # of a particular value at the index, values is the total number of values we have 
    entropy = 0
    for num in lst:
        if num != 0:
            temp = num/values * math.log2(num/values)
            entropy += temp
    return -1 * entropy
    

class DT: #Decision Tree 
    class Node:
        #start off with no features and no thresholds
        #find the feature and threshold that partitions the children using the functions
        def __init__(self, index, entropy=None):
            self.feature = None #the category that is used to categorize 
            self.threshold = None #this is the threshold to compare something. either = or not equal 
            self.index = index
            #left is less. right is greater than or equal
            self.entropy = entropy
            self.left = None 
            self.right = None 
            self.category = None #none means it is not a leaf node. a value is set for leaves

        def getEntropy(self, data): #this is a correct function 
            if self.entropy == None: #calculate entropy first
                count = Counter(data['class'].iloc[self.index])
                entropy = -1 * sum(count[key]/len(self.index) * math.log2(count[key]/len(self.index)) for key in count.keys())
                self.entropy = entropy
            return self.entropy     
        
        def __str__(self):
            lst = []
            lst.append("feature: " + str(self.feature))
            lst.append("threshold: " + str(self.threshold))
            lst.append("index: " + str(len(self.index)))
            lst.append("entropy: " + str("{:.2f}".format(self.entropy)) if self.entropy != None else "None")
            lst.append("category: " + str(self.category))
            return '   '.join(lst)
            
        def __repr__(self):
            return str(self)
            

    def __init__(self, data, end, classes):
        self.data = data
        self.root = self.Node([i for i in range(len(data))])
        self.end = math.ceil(len(self.data) * end / 100) #the point at which we don't classify anymore 
        #print("self.end is currently ", self.end)
        self.classes = classes
        self.makeTree()

    def makeTree(self):
        
        #I sorted and then i took something but should i have done the average instead of this current?
        #current implementation may have some bugs 
        def helper(node): ## call this on a node 
            currEntropy = node.getEntropy(self.data)
            
            best = ["feature", 0, 0, 0, 0] #feature, threshold, entropyleft, entropyright, colsubset
            
            if len(node.index) >= (2 * self.end + 2) and currEntropy != 0: 
                #print(" I am atleast getting in and trying to find best")
            
                for col in self.data.columns[: -1]: #choose features 
                    
                    #print(col)

                    colsubset = self.data.iloc[node.index][[col, self.data.columns[-1]]]
                    colsubset['original index'] = node.index
                    colsubset = colsubset.sort_values(by=col, ascending = True)
                    colsubset = colsubset.reset_index(drop=True)
                    
                    #print(colsubset.iloc[:30])

                    count = Counter(colsubset['class']) #get the classifications
                    
                    other_count = [0 for i in range(self.classes)]
                    for i in range(self.classes):
                        if i in count.keys():
                            other_count[i] = count[i]
                    
                    curr_count = [0 for i in range(self.classes)] #access the categories 

                    for i in range(len(colsubset)-2 - self.end): #choose threashold inclusive of self don't include last value 
                        location = colsubset['class'][i] 
                        
                        curr_count[location] += 1
                        other_count[location] -= 1
                        
                        
                        leftentro = calcEntropy(curr_count, i+1)
                        rightentro = calcEntropy(other_count, len(node.index) - i - 1)
                        
                        #weighted roundentropy 
                        roundentro = leftentro * (i+1 / len(node.index)) + rightentro * ((len(node.index) - i - 1)/ len(node.index))
                        
                
                        #if (roundentro) < currEntropy:
                            #print("this roundentro" , roundentro," < ", currEntropy)
            
                        #print(curr_count, other_count, roundentro, currEntropy)
                        #print("curr Entropy: ", currEntropy, "    and then roundentro is ", roundentro, "   col is ", col, "  while the i ", i)
                        
                        if ((roundentro) < currEntropy and (i + 1) > self.end and (len(node.index) - i - 1) > self.end):
                            #fits the nmin criteria
                            currEntropy = roundentro
                            best = [col, i, leftentro, rightentro, colsubset]
                            #print("new best should be ", best)
                #print(" this is my ultimate best ", best)
                
        
            if (best[0] == "feature"): #no improvement can be made through splitting 
                #or when we already reach end or entropy is 0 
                #print(" I am going to be a leaf since best is still feature ", node)
                
                subset = self.data.iloc[node.index][[self.data.columns[-1]]]
                count = Counter(subset['class'])
        
                count = [(count[key], key)for key in count.keys()]
                count = sorted(count, reverse=True)
                node.category = count[0][1]
                return 
            
            
            #final way to put in the entropy 
            #print(" i am not gonna be a leaf", node)
            colsubset = best[4]
            i = best[1]

            node.feature = best[0]
            
            node.threshold = colsubset[node.feature][i]

            node.left = self.Node(list(colsubset['original index'][:i+1]), best[2])
            node.right = self.Node(list(colsubset['original index'][i+1:]), best[3])
            

            #print(node)
            #print(node.left, node.right)
            
            helper(node.left)
            helper(node.right)

            #print(best)

        helper(self.root) #the call that started it all 


        
    def predict(self, pt): #predict the category of the value within the tree 
        #this is to traverse the tree 
        def helper(node, pt):
            if (node.category != None): #if it is a leaf 
                return node 
            
            options = list(self.data.columns[:-1]) #get the columns 

            ind = options.index(node.feature)
            if (pt[ind] <= node.threshold):
                return helper(node.left, pt)
            return helper(node.right, pt)
                
        node = helper(self.root, pt)
        #print(node)
        
        return int(node.category)

    def __repr__(self):
        lst = []
        def helper(node):
            #node.index  #this is to get the values of the node and then see what the issue is 
            #lst.append(str(self.data.iloc[node.index][['class']][:30]))
            lst.append(str(node))
            if node.left: helper(node.left)
            if node.right: helper(node.right)
            
        helper(self.root)
        return '\n'.join(lst)
        

### Preliminary Dataset Test

In [131]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

#iris df has 150 rows x 5 columns with continuous values on each 
#a different class is given a different number for encoding 

#print(iris_df['class'][1])
dt = DT(iris_df, 5, 3)
# dt.root.getEntropy()

print(dt)

predictme = iris_df.iloc[19]
print(predictme)
dt.predict(predictme)

feature: petal_length   threshold: 1.9   index: 150   entropy: 1.58   category: None
feature: None   threshold: None   index: 50   entropy: -0.00   category: 0
feature: petal_width   threshold: 1.4   index: 100   entropy: 1.00   category: None
feature: None   threshold: None   index: 32   entropy: -0.00   category: 1
feature: None   threshold: None   index: 68   entropy: 0.83   category: 2
sepal_length    5.1
sepal_width     3.8
petal_length    1.5
petal_width     0.3
class           0.0
Name: 19, dtype: float64


0

### Testing Function

In [132]:
def test(data, nmin, kfold, classes):
    #split the data set 
    skf = KFold(n_splits=kfold, shuffle=True, random_state=42)
    
    accurate = [0 for i in range(len(nmin))] #calculate a different accuracy for each nmin according to differnet splits 
    
    ##a;ldkfjs;ldkfjas;lkdfjsa;lkdfjasl;dfj have standard deviation as well please :) 
    
    loc = 1
    for train_index, test_index in skf.split(data):
        print("\n----------------------------------------------------\nSplit data starting ", loc, " iteration for k fold")
        
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        answer = [] 
        
        #print(data_train.iloc[:10])
    
        for j, nval in enumerate(nmin): #for each nmin 
            dt = DT(data_train, nval, classes) #create the tree    
            print(dt)
            
            for i in range(len(data_test)): #test each datapoint 
                lst = data_test.iloc[i]
                tempanswer = dt.predict(lst)
                answer.append(tempanswer)
                
            #print(data_test['class'][-30:])
            #print(answer[-30:])
            
            accuracy = accuracy_score(data_test['class'], answer)
            print(accuracy)
            print()
            accurate[j] += accuracy
            answer = []
        loc += 1
            
    accurate = [val / kfold for val in accurate]
    print(accurate)

### Testing on Iris Dataset. Using different Nmin 

In [133]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

nmin = [5, 10, 15, 20]
test(iris_df, nmin, 10, 3)


----------------------------------------------------
Split data starting  1  iteration for k fold
feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   category: None
feature: None   threshold: None   index: 44   entropy: -0.00   category: 0
feature: petal_length   threshold: 4.5   index: 91   entropy: 1.00   category: None
feature: None   threshold: None   index: 30   entropy: -0.00   category: 1
feature: None   threshold: None   index: 61   entropy: 0.78   category: 2
0.8666666666666667

feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   category: None
feature: None   threshold: None   index: 44   entropy: -0.00   category: 0
feature: petal_length   threshold: 4.5   index: 91   entropy: 1.00   category: None
feature: None   threshold: None   index: 30   entropy: -0.00   category: 1
feature: None   threshold: None   index: 61   entropy: 0.78   category: 2
0.8666666666666667

feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   categ

feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   category: None
feature: None   threshold: None   index: 44   entropy: -0.00   category: 0
feature: petal_width   threshold: 1.4   index: 91   entropy: 1.00   category: None
feature: None   threshold: None   index: 29   entropy: -0.00   category: 1
feature: None   threshold: None   index: 62   entropy: 0.80   category: 2
0.8

feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   category: None
feature: None   threshold: None   index: 44   entropy: -0.00   category: 0
feature: petal_width   threshold: 1.4   index: 91   entropy: 1.00   category: None
feature: None   threshold: None   index: 29   entropy: -0.00   category: 1
feature: None   threshold: None   index: 62   entropy: 0.80   category: 2
0.8

feature: petal_length   threshold: 1.9   index: 135   entropy: 1.58   category: None
feature: None   threshold: None   index: 44   entropy: -0.00   category: 0
feature: petal_width   threshold: 1.4   index:

### Testing on Spambase Dataset. Using different Nmin 

In [134]:
name = ["var_" + str(i) for i in range(57)] + ['class']

spambase_df = pd.read_csv("spambase.csv", names=name)

nmin = [5, 10, 15, 20, 25]

test(spambase_df, nmin, 10, 2)


----------------------------------------------------
Split data starting  1  iteration for k fold
feature: var_1   threshold: 0.0   index: 4140   entropy: 0.97   category: None
feature: None   threshold: None   index: 1965   entropy: -0.00   category: 0
feature: None   threshold: None   index: 2175   entropy: 0.82   category: 1
0.631236442516269

feature: var_1   threshold: 0.0   index: 4140   entropy: 0.97   category: None
feature: None   threshold: None   index: 1965   entropy: -0.00   category: 0
feature: None   threshold: None   index: 2175   entropy: 0.82   category: 1
0.631236442516269

feature: var_1   threshold: 0.0   index: 4140   entropy: 0.97   category: None
feature: None   threshold: None   index: 1965   entropy: -0.00   category: 0
feature: None   threshold: None   index: 2175   entropy: 0.82   category: 1
0.631236442516269

feature: var_1   threshold: 0.0   index: 4140   entropy: 0.97   category: None
feature: None   threshold: None   index: 1965   entropy: -0.00   cate

KeyboardInterrupt: 